# Chapter 3 Finding Null Space

In [1]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [3]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

We consider a matrix of size $10 \times 11$ of rank 7, which is non-square and not of full rank.

In [4]:
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [5]:
pivots = []
# LU-decomposition
P, L, U = sp.linalg.lu(A)
print(U)
# identify rank of U
rank_u = np.linalg.matrix_rank(U)
# find pivot entries
for i in range(rank_u):
    non_zero_indices = np.nonzero(U[i,:])
    pivots.append(non_zero_indices[0][0])
print('Pivot variables = ', np.int64(pivots))
# find column indices of free variables
free_variables = np.sort(list(set(range(n))- set(pivots)))
print("Free variables = ", free_variables)

[[-5.6326 -7.2932 -4.4209  1.1787  5.2399  1.7193 -7.9353 -1.8607  0.2294  1.1057 -1.3617]
 [ 0.      5.8976  1.908  -3.1187 -1.1355 -2.2901  1.4088 -1.601   0.344  -2.5084 -1.6704]
 [ 0.      0.     -4.3361 -0.6101 -1.8971 -2.7182 -4.8943  0.0058  2.1581 -3.6233 -0.5328]
 [ 0.      0.      0.      3.0336 -5.0298 -2.8854  6.8808 -0.729  -5.4434 -2.3809  3.5878]
 [ 0.      0.      0.      0.     -5.6616 -0.1132 -2.2355 -0.7585  1.561   3.8564 -0.567 ]
 [ 0.      0.      0.      0.      0.     -2.1323  5.5149 -5.1065  2.4937 -0.3244  2.0906]
 [ 0.      0.      0.      0.      0.      0.      0.0944 -2.0777 -1.2016  1.2656 -0.0775]
 [ 0.      0.      0.      0.      0.      0.      0.     -0.     -0.      0.     -0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]]
Pivot variables =  [0 1 2 3 4 5 6]
Free variables =  [ 7  8  9 10]


Note that we invoke **numpy.linalg.matrix_rank** which might use advanced results not covered in until Chapter 3 of our textbook. We will implement this functionality from the scratch in **Ch3-3 Solving Non-invertible Linear System**. 

Collect pivot columns of U into **U_pivot**, and columns for free variables into **U_free**. Then solve $U_{pivot} X = -U_{free}$ where $X$ is an $n \times (n- Rank A)$ matrix solution of the equation.

In [6]:
# Collect pivot columns and free variables
U_pivot = U[:rank_u, pivots]
print('pivot columns of U = ')
print(U_pivot)
U_free = U[:rank_u, free_variables]
print('free variable columns of U = ')
print(U_free)
# Solve for the pivot part of a basis of the null space of A
X_pivot = np.linalg.solve(U_pivot,-U_free)
print('pivot part of X = ')
print(X_pivot)
# Combine pivot part and free variable part
X = np.eye(n)[:,free_variables]
X[pivots,:] = X_pivot

print('X = ')
print(X)

np.allclose(A@X, 0)

pivot columns of U = 
[[-5.6326 -7.2932 -4.4209  1.1787  5.2399  1.7193 -7.9353]
 [ 0.      5.8976  1.908  -3.1187 -1.1355 -2.2901  1.4088]
 [ 0.      0.     -4.3361 -0.6101 -1.8971 -2.7182 -4.8943]
 [ 0.      0.      0.      3.0336 -5.0298 -2.8854  6.8808]
 [ 0.      0.      0.      0.     -5.6616 -0.1132 -2.2355]
 [ 0.      0.      0.      0.      0.     -2.1323  5.5149]
 [ 0.      0.      0.      0.      0.      0.      0.0944]]
free variable columns of U = 
[[-1.8607  0.2294  1.1057 -1.3617]
 [-1.601   0.344  -2.5084 -1.6704]
 [ 0.0058  2.1581 -3.6233 -0.5328]
 [-0.729  -5.4434 -2.3809  3.5878]
 [-0.7585  1.561   3.8564 -0.567 ]
 [-5.1065  2.4937 -0.3244  2.0906]
 [-2.0777 -1.2016  1.2656 -0.0775]]
pivot part of X = 
[[-16.3422 -10.7317   9.7811  -1.0556]
 [ 23.7737  17.6184 -14.1053   1.5818]
 [-52.6578 -32.3368  31.9114  -2.6544]
 [-14.2496  -3.6552   9.1264  -0.8987]
 [ -9.9102  -5.4295   6.6684  -0.4861]
 [ 54.5046  34.0776 -34.8114   3.1021]
 [ 22.      12.7238 -13.4009   0.82

True

Check the linear independence of columns of matrix $X$ since, after row shuffling, a submatrix of $X$ is an identity matrix.